In [1]:
## 1.a All observation has the same intial weight as 1/(number of observation) which is 1/100

## 1.b Because the observation is misclassified by the first learner, the new weight (second round)
#      will be larger than the initial weight. Subsequently, the next learner will focus on this
#      observation to find way to correctly classify it.

In [2]:
## 2. AdaBoost.M1 is an ensemble learning algorithy be cause it builds and uses more than 1 learner.
#     The framework sequentially build many learners to correctly classify the most number of
#     observations, until reaching a preset number of learners. Finally, the prediction is a
#     combined result of all learners built.

In [3]:
## 3. Observation      Old Weight      Correct?      New Weight
#         1               0.16         Correct
#         2               0.64         Correct
#         3               0.08         Incorrect
#         4               0.12         Incorrect
#     
#     epsilon = (0.08 + 0.12) / (0.16 + 0.64 + 0.08 + 0.12) = 0.2    < 0.5
#     alpha = log[(1 - 0.2) / 0.2] = 1.386
#     eta = 0.1
#
#     new weight 1 = 0.16 * e^(0.1 * 1.386 * (-1)) = 0.139 0.15
#     new weight 2 = 0.64 * e^(0.1 * 1.386 * (-1)) = 0.557 0.6
#     new weight 3 = 0.08 * e^(0.1 * 1.386 * 1) = 0.092    0.1
#     new weight 4 = 0.12 * e^(0.1 * 1.386 * 1) = 0.138    0.15

In [4]:
## 4. If an AdaBoost ensemble underfits the training dataset, the first thing that I would do is
#     to tweak the learning rate. Lower learning rate should help adapt to the dataset better.
#     Lower learning rate with a small number of learners could lead to underfitting as well, so
#     that would be the next thing I will check.

In [5]:
## 5. c

In [1]:
## 6.a
import boto3
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
from sklearn.svm import SVC

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'danhtran358-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining input and target variables
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Scale features to range 0-1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [3]:
md1 = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
pred1 = md1.predict_proba(X_test)[:,1]
pred1 = np.where(pred1 < 0.1, 0, 1)
recall_score(Y_test, pred1)

0.9017857142857143

In [5]:
md2 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)
pred2 = md1.predict_proba(X_test)[:,1]
pred2 = np.where(pred2 < 0.1, 0, 1)
recall_score(Y_test, pred2)

0.9821428571428571

In [ ]:
md3 = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)
pred3 = md1.predict_proba(X_test)[:,1]
pred3 = np.where(pred3 < 0.1, 0, 1)
recall_score(Y_test, pred3)

In [27]:
def get_recall_rf(X_train, X_test, Y_train, Y_test):
    ## Random forest model
    md_rf = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_rf = md_rf.predict_proba(X_test)[:,1]
    pred_rf = np.where(pred_rf < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_rf)

In [28]:
def get_recall_ada_tree(X_train, X_test, Y_train, Y_test):
    ## Adaboost model with Decision Tree leaners
    md_ada_tree = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_ada_tree = md_ada_tree.predict_proba(X_test)[:,1]
    pred_ada_tree = np.where(pred_ada_tree < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_ada_tree)

In [29]:
def get_recall_ada_svc(X_train, X_test, Y_train, Y_test):
    ## Adaboost model with Support Vector Machine leaners
    md_ada_svc = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_ada_svc = md_ada_svc.predict_proba(X_test)[:,1]
    pred_ada_svc = np.where(pred_ada_svc < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_ada_svc)

In [30]:
print(get_recall_rf(X_train, X_test, Y_train, Y_test))
print(get_recall_ada_tree(X_train, X_test, Y_train, Y_test))

0.9285714285714286
1.0


In [26]:
print(get_recall_ada_svc(X_train, X_test, Y_train, Y_test))

0.9196428571428571


In [31]:
print(get_recall_ada_svc(X_train, X_test, Y_train, Y_test))

1.0


In [11]:
a = list()
b = list()
a.append(1)
a.append(2)
a.append(3)
a.append(4)
b.append(1)
b.append(2)
b.append(3)
b.append(4)
print(a)
print(b)
c = a + b
print(c)

In [4]:
rf_recall = list()
ada_tree_recall = list()
ada_svc_recall = list()
scaler = MinMaxScaler()

for i in range(2):
    ## Defining input and target variables
    X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
    Y = heart['TenYearCHD']

    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    ## Scale features to range 0-1
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

    ## Random forest model
    md_rf = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)

    ## Predict and classfify
    pred_rf = md_rf.predict_proba(X_test)[:,1]
    pred_rf = np.where(pred_rf < 0.1, 0, 1)

    ## Calculate recall score and store
    rf_recall.append(recall_score(Y_test, pred_rf))

    ## Adaboost model with Decision Tree leaners
    md_ada_tree = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

    ## Predict and classfify
    pred_ada_tree = md_ada_tree.predict_proba(X_test)[:,1]
    pred_ada_tree = np.where(pred_ada_tree < 0.1, 0, 1)

    ## Calculate recall score and store
    ada_tree_recall.append(recall_score(Y_test, pred_ada_tree))

    ## Adaboost model with Support Vector Machine leaners
    md_ada_svc = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)

    ## Predict and classfify
    pred_ada_svc = md_ada_svc.predict_proba(X_test)[:,1]
    pred_ada_svc = np.where(pred_ada_svc < 0.1, 0, 1)

    ## Calculate recall score and store
    ada_svc_recall.append(recall_score(Y_test, pred_ada_svc))
    
print(rf_recall)
print(ada_tree_recall)
print(ada_svc_recall)

[0.9285714285714286, 0.9196428571428571]
[1.0, 1.0]
[1.0, 1.0]


In [ ]:
rf_recall = list()
ada_tree_recall = list()
ada_svc_recall = list()
scaler = MinMaxScaler()

for i in range(2):
    ## Defining input and target variables
    X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
    Y = heart['TenYearCHD']

    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    ## Scale features to range 0-1
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    rf_recall.append(get_recall_rf(X_train, X_test, Y_train, Y_test))
    ada_tree_recall.append(get_recall_ada_tree(X_train, X_test, Y_train, Y_test))
    ada_svc_recall.append(get_recall_ada_svc(X_train, X_test, Y_train, Y_test))
    
print(rf_recall)
print(ada_tree_recall)
print(ada_svc_recall)